In [ ]:
import pandas as pd
import pickle
import requests
import json
import re
from tqdm import tqdm
from bs4 import BeautifulSoup

## url pickle load
pickles = ['감성_에세이?q=g']


writer_list = []
for file in pickles:
    with open(file[:-4]+"_userId.txt","rb") as fr:
        writers = pickle.load(fr)
    writer_list.append(writers)  ## [[카테고리1 게시글 url...],[카테고리2 게시글 url], ....[카테고리24 게시글 url]]

## 게시글 속 정보 수집
def def_craw(writer):
    
    json_data = {}
    data = []
    res_text = []
    tag_keyword=[]
    
    tag_title,tag_nickname,tag_publish_date,tag_url,tag_url_plink = None,None,None,None,None
    tag_share,tag_like = str,str
    for idx,url in enumerate(writer):
        if idx % 500 == 0 : print("전체",len(writer),"중에",idx)
        if res_text == []: # 첫 시작에러 방지
            pass
        else :
            # to json
            json_data['title'] = tag_title  
            json_data['nickname'] = tag_nickname
            json_data['publish_date'] = tag_publish_date
            json_data['keyword'] = tmp_keyword   
            json_data['like'] = tag_like # like 없는 경우 ''
            json_data['share'] = tag_share # share 없는 경우 None            
            json_data['comment'] = tag_comment # comment 없는 경우 ''
            json_data['url'] = tag_url
            json_data['url_plink'] = tag_url_plink 
            json_data['text'] = res_text

        data.append(json_data)
        
        json_data = {} # 누적방지 초기화
        tmp_keyword = [] # 누적방지 초기화
        res_text = [] # 누적방지 초기화 
        
        # beautifulsoup
        html = requests.get('https://brunch.co.kr{text_url}'.format(text_url=url))
        soup = BeautifulSoup(html.text, 'html.parser')
        
        if soup.find('title').text == "brunch":
            pass
        else:
            tag_title = soup.find('title').text # 게시글 title
            tag_url = soup.find("meta",property='og:url')['content'] # 게시글 본주소
            tag_nickname = soup.find("meta",{'name':'article:media_name'})['content'] # 작가 nickname
            tag_url_plink = soup.find("meta",property='dg:plink')['content'] # 암호주소? # 모바일?
            tag_publish_date = soup.find("meta",property='article:published_time')['content'] # 발행일
            tag_keyword = soup.find_all('a',href=re.compile('/keyword')) # 게시글 키워드
            tag_like = soup.find('span',{'class':'f_l text_like_count text_default text_with_img_ico ico_likeit_like #like'}) #좋아요 수
            tag_share = soup.find('span',{'class':'f_l text_share_count text_default text_with_img_ico'}) # 공유 수
            tag_comment = soup.find('span',{'class':'f_l text_comment_count text_default text_with_img_ico'}) # 댓글 수
            text_h4 = soup.find_all(class_='wrap_item item_type_text')
            
            for text in text_h4:
                res_text.append(text.text)
    
            if tag_like == None:
                tag_like = "0"
            else:
                tag_like = tag_like.text # 좋아요 수

            if tag_share == None:
                tag_share == "0"
            else:
                tag_share = tag_share.text # 공유 수

            if tag_comment == None:
                tag_comment =="0"
            else:
                tag_comment = tag_comment.text

            for keyword in tag_keyword:
                tmp_keyword.append(keyword.text)
                
    return data ## 수집한 정보를 담은 dictionary로 반환

categories = ['지구한바퀴_세계여행','시사_이슈','IT_트렌드',
                '취항저격_영화리뷰','오늘은_이런책','뮤직_인사이드',
                 '직장인_현실조언','스타트업_경험담','육아_이야기',
                 '요리_레시피','건강_운동','멘탈관리_심리탐구',
                 '문화_예술','인문학_철학','쉽게_읽는_역사',
                '우리집_반려동물','사랑_이별','감성_에세이']


## 카테고리 -> 게시글의 순서로 2차 크롤링 진행
## 카테고리별로 정보를 담은 json 형식으로 저장(총 24개 json file)
from collections import OrderedDict
for idx,writer in enumerate(writer_list):
    to_json = None
    data = def_craw(writer) # 2단계 크롤링 실행
    
    del data[0]
    del data[0]
    
    to_json = OrderedDict()
    to_json['name'] = categories[idx] # category name
    to_json['version'] = "2020-06-01"
    to_json['data'] = data
    
    with open(categories[idx]+".json","w") as make_file:
        json.dump(to_json,make_file)

In [1]:
import pandas as pd
import pickle
import requests
import json
import re
from tqdm import tqdm
from bs4 import BeautifulSoup

In [10]:
html = requests.get('https://brunch.co.kr/@pubss/175')
soup = BeautifulSoup(html.text, 'html.parser')

text_h4 = soup.find_all(class_='wrap_item item_type_text')
res_text=[]
for text in text_h4:
    res_text.append(text.text)
    
str(res_text)

import re
def pre_text_2(x):
    pa = re.compile("^\\\\xa0|xa") # xla 등 불용어 제거
    pa1 = re.compile(r"'http.*?'") # 전체 url 제거
    pa2 = re.compile(r'\([^)]*\)') # () 사이 문자 
    pa3 = re.compile('[^\w\s]') # 특수문자 삭제
    pa4 = re.compile(r'[^a-zA-Zㄱ-힗]') # 한글,영어만 남김

    x = re.sub(pa,' ',x)
    x = re.sub(pa1,' ',x)
    x = re.sub(pa2,' ',x)
    x = re.sub(pa3, ' ',x)
    x = re.sub(pa4, ' ',x)
    x = x.strip()
    x = " ".join(x.split())
    return x 

pre_text_2(str(res_text))

'낯선 곳에서 맞이하는 아침은 언제나 상쾌하다 새벽 늦게 수다를 떨다 잠들어도 밤새 술을 마시다 잠들어도 집 침대가 아닌 곳에서 일어나는 아침은 마치 영화처럼 지저귀는 새소리와 함께 단번에 침대에서 나올 이 생긴다 쳇바퀴 굴리는 삶이라 일어나면 어제와 같은 하루가 펼쳐질 것이라는 것을 아는 권태에 가득 찬 아침 침대에서 분만 분 만을 중얼거리는 것과는 딴 판이다 눈을 뜨면 무슨 하루가 펼쳐질까 궁금하니 일어나는 것도 즐거웠다 눈을 뜨고 씻기도 전에 게스트 하우스의 식당으로 가서 짜이 한 잔을 마셨다 아침 일찍 일어나 피곤에 찌들어 커피를 한 잔 마시며 잠에서 깨는 것이 일상이었으나 인도에서는 이 짜이라는 차가 커피를 대신하게 되었다 인도에 왔으면 인도의 룰을 따라야 했다 짜이 한 잔과 함께 게스트하우스를 오가는 사람들에게 눈인사를 나누는 동안 모스크의 아침 종소리가 맨발을 통해 진동으로 울려 퍼진다 대충 차려 입고 바라나시를 구경하러 밖으로 나왔다 무너질 듯 아슬아슬한 회색 골목을 지나면 화려한 색상들의 상점들이 분주하다 아침부터 요란스러운 오토바이 소리와 하품하는 개 아직 쓰레기 더미 사이에서 잠이 든 소를 지나치면 이 골목 사랑방인 짜이 집이 나왔다 오늘도 역시 많은 인도인들이 이 짜이 집 앞에서 황토색 컵에 찰랑거리는 짜이를 담고 수다를 떨고 있다 계급 지위 민족을 막론하고 인도인들을 하나로 묶어주는 것은 바로 이 짜이다 우리나라 사람들을 하나로 묶어 주는 것은 무엇일까 한국이라는 문화 DNA가 모든 한국인에게 발현되어 하나로 묶어주는 것은 다름이 아니라 김치일 것이다 경상도 사람이든 전라도 사람이든 혹은 이북에 살든 한국 문화의 DNA가 하나로 통하는 것은 김치다 다른 사람들은 우리를 하나로 묶는 문화 DNA는 아리랑이나 한 혹은 정 이라고 말한다 하지만 아리랑을 우리를 묶어준다고 하기에는 현대 한국을 사는 사람들에겐 큰 감흥이 오지 않는다 낯설지는 않지만 그렇다고 평소에도 즐겨 들으며 아리랑을 부르지는 않는다 한은 추상적이다 우리가 직접 체감하기 어려